<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">

# AST4310 2023, Project 3

</div>

Authored by: *(Replace this text with your candidate numbers)*.

In [1]:
# Your imports here

### Exercise 1: Non-LTE Radiative Transfer with the Feautrier method and $\mathbf{\Lambda}$ iteration [40 points]

The Feautrier method consists of solving the equation

\begin{equation}
\mathbf{T}P = S,
\end{equation}

where $\mathbf{T}$ is a tridiagonal matrix. The function `Tmatrix` computes $\mathbf{T}$ for a single frequency and direction on a 1D plane-parallel atmosphere, given an optical depth scale and $\mu$. Below we use the C/Python convention that 0 is the first index of an array, and $n-1$ the last point.

The source function for a two-level atom with coherent scattering can be written as

\begin{equation}
S = (1-\varepsilon)\mathbf{\Lambda}[S] + \varepsilon B,
\end{equation}
and a direct solution can be written as
\begin{equation}
S = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda})^{-1}[\varepsilon B],
\end{equation}

which involves inverting matrices (impractical for most real-life applications). Other numerical methods to solve for $S$ are the classical $\Lambda$ iteration:

\begin{equation}
S^{(n+1)} = (1-\varepsilon)\mathbf{\Lambda}[S^{(n)}] + \varepsilon B,
\end{equation}

where we need a guess for $S^{(1)}$, often $S^{(1)}=B.$ An alternative that is much more efficient is the approximate (or accelerated) $\Lambda$ iteration:

\begin{equation}
S^{(n+1)} = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda}^*)^{-1}\left[S^\mathrm{FS}-(1-\varepsilon)\mathbf{\Lambda}^*[S^{(n)}]\right].
\end{equation}

For the iterative schemes, we can measure the convergence speed by calculating the fractional difference between source functions of successive iterations, e.g.:
\begin{equation}
\delta = \left|\left|\frac{S_\nu^{(n+1)}-S_\nu^{(n)}}{S_\nu^{(n)}}\right| \right|.
\end{equation}

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[5 points]* At the top of the atmosphere, we can assume $I^-=0$. Why? Using a Taylor expansion around the Feautrier $P_0$ and assuming $I^-=0$ at the top of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the top can be written as:

$$
\begin{aligned}
B_0 &= \frac{2}{\Delta\tau^2} + \frac{2}{\Delta\tau} + 1, \\ 
C_0 &= \frac{2}{\Delta\tau^2}, \\
\Delta\tau &=\tau_{1} - \tau_{0}.
\end{aligned}
$$ 

* *[5 points]* At the bottom of the atmosphere, we can assume $I^+=S$. Why? Using a Taylor expansion around the Feautrier $P_{n-1}$, and assuming $I^+=S$ at the bottom of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the bottom can be written as:

$$
\begin{aligned}
A_{n-1} &= \frac{2}{\Delta\tau(\Delta\tau+2)},  \\ 
B_{n-1} &= \frac{2+2\Delta\tau +\Delta\tau^2}{\Delta\tau(\Delta\tau+2)},\\
\Delta\tau &=\tau_{n-1} - \tau_{n-2}.
\end{aligned}
$$

* *[5 points]* Using the function `lambda_matrix()` and `tau = numpy.logspace(-4, 2, 50)`, plot the $\mathbf{\Lambda}$ matrix and discuss its physical meaning. Explore the effect of using a different number of points in the Gaussian quadrature. What is a reasonable number of points?

* *[5 points]* The function `solve_cs_direct()`, which takes as arguments $\tau$, $B$, and $\varepsilon$, computes $S$ and $J$ using a direct solution for the problem of coherent scattering in a two-level atom (using matrix inversions). Use it to solve the problem for $\varepsilon=10^{-3}$, `tau = numpy.logspace(-4, 4, 50)` and $B=1.5\tau$. Do a log-log plot for S, J, and B. In this case, why is $J>B$ at the surface?
    
* *[8 points]* Write a function `solve_cs_CLI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the classical $\Lambda$ iteration (CLI). For the simple case where $B=1$ everywhere, $\varepsilon=10^{-2}$, plot $S$, $J$, $B$, for CLI and the direct solution from `solve_cs_direct()`. Does CLI work well for this case? How many iterations do you need to achieve $\delta < 10^{-3}$?
    
* *[12 points]* Write a function `solve_cs_ALI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the accelerated $\Lambda$ iteration (ALI). For the $\mathbf{\Lambda}^*$ operator, use the OAB operator, which simply uses the diagonal of the full $\mathbf{\Lambda}$ operator. Usually, this $\mathbf{\Lambda}^*$ is computed without having to build the $\mathbf{\Lambda}$ matrix explicitly, during the tridiagonal solution of $\mathbf{T}P = S$. Use the provided function `diag_inverse_tri()` to obtain $\mathbf{\Lambda}^*$ from the inverse of $\mathbf{T}$ matrices (as before, account for different angle quadratures), using `Tmatrix(tau, mu, format='banded')` to get $\mathbf{T}$ in the correct format. For the simple case where $\varepsilon=10^{-5}$, `tau = numpy.logspace(-4, 4, 50)`, `B = tau[::-1] * 1.5` , plot $S$ and $B$, for $S$ computed with the direct solution, CLI, and ALI, both with a maximum of 100 iterations. Discuss the differences. How many iterations of ALI do you need to be close to the direct solution? And how many iterations of CLI?
    
</div>

In [2]:
# Your answers here

### Exercise 2: Zeeman effect and Polarisation [15 points]

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[3 points]* Assume that the terms $^5P$ and $^5D$ can define respectively the upper and lower level of a bound-bound transition. How many different levels does each term have, and how many permited transitions are possible with Zeeman splitting? Remember that a level is set by a fixed S, L, and J. The quantum mechanical selection rules for electric dipole transitions are $\Delta M_j=0, \pm 1$ and $\Delta J = 0, \pm 1$ (as long as $J^u$ and $J^l$ are not both zero). 
* *[3 points]* Calculate the effective Landé factor $\bar{g}$ for the $^5 P_1 - {}^5D_1$ transition. You will need this value in exercise 3 for the Ti I 2.221 $\mu$m line.
* *[4 points]* Using the definitions of the Stokes profiles, show that we always have $I^2 \ge Q^2 + U^2 + V^2$.
* *[5 points]* What happens to RTE solutions when you change $\chi$ by 180º? And when you change $\gamma$ by 180º?

</div>

In [3]:
# Your answers here

### Exercise 3: Milne-Eddington lines from the FALC model [30 points]

In this exercise, you are going to test the validity of the Unno-Rachkovsky solution and the Milne-Eddington approximation. Is it really a good fit for lines in stellar atmospheres? You will work with a neutral titanium line observed in the infrared, with the following properties:

|                  |  Ti I 2221 |
|------------------|-----------:|
| Air wavelength (nm)  | 2221.728  |
| Lower level      | 4s $^5P_1$ |
| Upper level      | 4p  ${}^5D_1$|
| $\chi_{1,l}$ (aJ)| 0.27774      |
| $\chi_{1,u}$ (aJ)| 0.36715 |
| $\gamma_\mathrm{rad}$ (s$^{-1}$) | 7.5e4 |
|   $g_l$          | 3      |
|   $g_u$          | 3      |
| $f_{lu}$         | 5.55e-3  |
| Solar Ti abundance  | 4.95     |
| Atomic mass (u)  | 47.867 |
| $\alpha$         | 0.253 |
| $\sigma$         | 289.8 |

Here $g_l$ and $g_u$ denote the statistical weights, not the Landé factors. For some of the questions below you will need to compute the disk-centre intensity for the line using the FALC model, in LTE, similar to what you did in Project 2 (feel free to reuse your previous code). The partition function for Ti I is given in the background notebook (assume no more states beyond Ti III).

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[3 points]* Does the Ti I 2.221 $\mu$m line exhibit normal or anomalous Zeeman effect?
    
* *[5 points]* Modify the function `unno_rachkovsky()` so that it works for arbitrary Zeeman patterns of the anomalous Zeeman effect.
    
* *[14 points]* Compute the Ti I 2.221 $\mu$m disk-centre intensity from the FALC model for the unpolarised case in LTE, using the procedure from Project 2. Then compare with the profile using the Unno-Rachkovsky solution (without magnetic field, B=0). To obtain the mean quantities for the Unno-Rachkovsky solution, you will need to look at the different quantities in the formation range of the spectral line. For $S_0$ and $S_1$, do a linear fit to $B_\lambda(T)$ vs $\tau_\mathrm{c}$ in the region where $3\times 10^{-5} \lesssim \tau_\mathrm{c} \lesssim 3$. For the other quantities such as $\Delta\lambda_D$ you can take average or typical values in that region (but do note that regions where $\tau_\mathrm{c} \gtrsim 1$ are more important). Based on this comparison, would you say the Unno-Rachkovsky solution is appropriate for the Ti I 2.221 $\mu$m line with the FALC model?
    
* *[8 points]* Now redo the Unno-Rachkovsky calculation assuming a vertical magnetic field for the Ti I 2.221 $\mu$m line with the parameters found in the previous question. Include all Zeeman components. What value of B would you need to see the split between $\sigma_r$ and $\sigma_b$ in the intensity profile? What value of B would you need to see the split among the individual $\sigma_b$ components (not just between $\sigma_r$ and $\sigma_b$)?


*Hints:* In the absence of magnetic field, Ti I line should not show a central reversal. You can adjust slightly (within a factor of a few) $S_0$ and $S_1$ for the Unno-Rachkovsky solution, to get a line profile closer to the full LTE solution.
</div>

In [4]:
# Your answers here